# Librairies


In [6]:
import numpy as np 
import matplotlib.pyplot as plt 
import time

# Préalable

## Initialisation

In [3]:
def initialize(ni,ri):
    Ui = np.zeros((ni, ri))
    indices = np.array_split(np.arange(ni), ri)
    for j, idx in enumerate(indices):
        Ui[idx, j] = 1 / np.sqrt(len(idx))

    return Ui

def initialize_G(r1,r2,r3):
    G= np.random.rand(r1,r2,r3)

    return G

In [5]:
G= initialize_G(10,20,30)
tt=G[:,:,0]
print(tt.shape)

(10, 20)


In [ ]:
U1=initialize(n1,r1)
U2=initialize(n2,r2)
U3=initialize(n3,r3)

## Calcul de l'erreur

In [ ]:
def etalT(T):
    n1, n2, n3 = T.shape
    T_prime=T.reshape(n1, n2 * n3)
    return T_prime

def calcul_error(T, G, U1, U2, U3):
    M_sum=U3[0,0] @ G[:,:,0]
    for i in range(1, G.shape[2]):
        M_sum+=U3[0,i] @ G[:,:,i]
    M_sum = U1 @ M_sum @ U2.T
    Mat=np.bmat(M_sum)

    for i in range (1, U3.shape[0]):
        M_sum=U3[i,0] @ G[:,:,0]
        for j in range(1, G.shape[2]):
            M_sum+=U3[i,j] @ G[:,:,j]
        M_sum = U1 @ M_sum @ U2.T
        Mat=np.bmat([[Mat],[M_sum]])
    
    T_prime=etalT(T)
    error=np.linalg.norm(T_prime-Mat)/np.linalg.norm(T_prime)
    return error

## Optimisation

In [ ]:
def optiU(T,U,U2,U3,G):
    T_=etalT(T)

    n1=T.shape[0]
    n2=T.shape[1]
    n3=T.shape[2]

    r1=G.shape[0]
    r2=G.shape[1]
    r3=G.shape[2]

    for i in range (n1):
        best_position=0
        best_value=float('inf')

        for j in range (r1):

            for p in range (r1):
                U[i,p]=0
            U[i,j]=1
            value=calcul_error(T, G, U, U2, U3)
            if value<best_value:
                best_value=value
                best_position=j
        U[i,best_position]=1
    


    return U,best_value


def optiG(T,U1,U2,U3,G):
    
    
    
    return G

In [ ]:
def Optimization(T, U1, U2, U3, G, max_iter=10,maxtime=300,tol=1e-6):
    erreurU1=[]
    erreurU2=[]
    erreurU3=[]
    erreurG=[]
    start_time=time.time()
    for i in range (max_iter):
        U1,err=optiU(T,U1,U2,U3,G)
        erreurU1.append(err)

        U2,err=optiU(T,U2,U1,U3,G)
        erreurU2.append(err)

        U3,err=optiU(T,U3,U1,U2,G)
        erreurU3.append(err)
        
        G,err=optiG(T,U1,U2,U3,G)
        erreurG.append(err)

        error=calcul_error (T,G,U1,U2,U3)
        if error<tol:
            print("Seuil de tolérance atteint")
            break
        if time.time()-start_time>maxtime:
            print("Temps maximum écoulé")
            break
        print(f"itération {i+1} : erreur={error}")
    return U1,U2,U3,G,erreurU1,erreurU2,erreurU3,erreurG

## Print

In [10]:
def print_results(erreurU1,erreurU2,erreurU3,erreurG):
    plt.plot(erreurU1, label='Erreur U1')
    plt.plot(erreurU2, label='Erreur U2')
    plt.plot(erreurU3, label='Erreur U3')
    plt.plot(erreurG, label='Erreur G')
    plt.yscale('log')
    plt.xlabel('Itérations')
    plt.ylabel('Erreur')
    plt.title('Convergence des erreurs')
    plt.legend()
    plt.show()

# Algo

Variables

In [ ]:
max_iter=100
tol=1e-6
maxtime=300

In [ ]:
T=np.random.rand(60,80,100)
n1,n2,n3=T.shape
r1=5
r2=10
r3=15
U1=initialize(n1,r1)
U2=initialize(n2,r2)
U3=initialize(n3,r3)
G=initialize_G(r1,r2,r3)

In [ ]:
U1,U2,U3,G,erreurU1,erreurU2,erreurU3,erreurG=Optimization(T, U1, U2, U3, G, max_iter,maxtime,tol)
print_results(erreurU1,erreurU2,erreurU3,erreurG)